<a href="https://colab.research.google.com/github/jcj402-sys/quantum_algorithm/blob/main/shor_factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Shor's Factorization Algorithm
---

이 노트북에서는 Shor's Factorization Algorithm을 구현하는 실습을 진행할 것입니다. Basic Quantum operator의 리뷰부터 시작해서, 15의 소인수분해를 진행해보도록 하겠습니다.

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
import numpy as np
import math
import matplotlib.pyplot as plt
from fractions import Fraction
from collections import Counter

In [ ]:
#문제 1. 15의 약수 찾기 정답: 3, 5

In [10]:
def c_amod15(a:int, power:int): #3,4,5,6 과정에서 답이 아니라는 것을 알림
  if a not in [2,4,7,8,11,13]:
    raise ValueError(f"a를 다시 골라주세요")
  U=QuantumCircuit(4,name=f"{a}^{power} mod 15")
  for i in range(power):
    if a in [2,13]:
      U.swap(0,1) #0, 1 state change
      U.swap(1,2)
      U.swap(2,3)
    if a in[7,8]:
      U.swap(2,3)
      U.swap(1,2)
      U.swap(0,1)
    if a in[4,11]:
      U.swap(1,3)
      U.swap(0,2)
    if a in[7,11,13]:
      for q in range(4): U.x(q)
  return U.to_gate().control()

In [8]:
def estimate_person_with_circuit_and_plot(N:int, a:int, shots: int =100):
"""
회로 생성
회로도 출력
qpe
히스토그램
후보 주기 출력
"""
g=math.gcd(a,N)
if g==1:
    print("답 발견")
    return None, [g,N//g], None
n=4#타겟 레지스터
t=8#카운팅 레지스터

qc=QuantumCircuit(t+n,t)
for q in range(t): qc.h(g)
qc.x(t)
for q in range(t):
    qc.append(c_amod15(1,2**q), [q]+[t+i for i in range(n)])

qc.append(QFT(t,inverse=True), range(t))

qc.measure(range(t), range(t))

display(qc.draw(fold=25))

backend=AerSimulator()
job=bachend.run(transpile(qc, backend), shots=shots)
counts=job.result().get_counts()

display(plot_histogram(counts, title=f"qpe results (a={a}, N={N})"))

top3 Counter(counts).most_common(3)

print("toq 3 measurement: ")
candidates=set()
for bits, cnt in top3:
  val=int(bitsd,2)
  phase=val/(2**t)
  print(f"{bits} ({cnt}회 --> phase = {phase:5f}")
  if val==0: continue
  frac=Fraction(phase).limit_demonator(N)
  r=fracdenominator
  if r>1:
    candidates.add(r)
  if not candidates:
    print("유효 r 못찾음")
    return None, None, qc
  print(f"후보 주기 r={sorted(candidates)}")
  return list(candidates), None, qc

IndentationError: expected an indented block after function definition on line 1 (ipython-input-274022651.py, line 2)

In [ ]:
from ast import Nonlocal
def compute_factors(N:int, a:int, R:int):
  if r%2 !=0:
    print(f"r={r}은 홀수>실패")
    return None
  exp=pow(a,r//2,N)
  f1=math.gcd(exp+1,N)
  f2=math.gcd(exp-1,N)


In [12]:
def run_shor_full_visual(N:int, a:int, shots:int=100):
  r_list, factors, circuit=estimate